In [ ]:
# Install openai
!pip install openai

import pandas as pd
import os     
from openai import OpenAI
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY")

# Set the environment variable (do NOT hardcode the key in the notebook)
os.environ["OPENAI_API_KEY"] = secret_value_0

# Check whether API_KEY is
print("OPENAI_API_KEY loaded:", bool(os.environ.get("OPENAI_API_KEY")))

# Create the OpenAI client (automatically read OPENAI_API_KEY)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
## Medium post
# https://medium.com/@2hyowon/from-natural-language-to-sql-in-seconds-a-lightweight-workflow-37ecb2ae5433

In [ ]:
# file_path example (work on Kaggle Notebook)
file_path = '/kaggle/input/adidas-sales-dataset/Adidas US Sales Datasets.xlsx'

# Existence check (helps catch path typos early)
assert os.path.exists(file_path), f"File not found: {file_path}"

# Read the Excel file (Read dataset with skipping rows and dropping the first row due to the dataset format)
# - skiprows=range(1, 4): skip extra header/notes rows if the file has them
# - header=1: treat the second row as the column header
df = pd.read_excel(file_path, skiprows=range(1, 4), header=1)
df = df.iloc[: , 1:]     # Drop the first column

# Quick shape check (rows, columns)
print("DataFrame shape:", df.shape)

# Preview the first few rows to confirm the data loaded correctly
df.head()

# Replace spaces with underscores so columns are easier to reference in SQL
df.columns = [col.replace(" ", "_") for col in df.columns]

# Display column names to verify the cleanup
print("Columns:", df.columns.tolist())

# Check data types (useful before inserting into a database)
df.info()

# Example Pandas Query for Sum of Sales by Region 
sales_by_region = (
    df.groupby("Region")["Total_Sales"]
      .sum()
      .sort_values(ascending=False)
)

print("Total Sales by Region (descending):")
print(sales_by_region)


In [ ]:
import sqlite3

temp_db = sqlite3.connect(":memory:")
df.to_sql(name="Sales", con=temp_db, if_exists="replace", index=False)

In [ ]:
def build_schema_prompt(df):
    """
    Build a minimal schema prompt for the LLM so it knows the table name and columns.
    """
    # Join column names into a single comma-separated list
    cols = ", ".join(df.columns)

    prompt = f"""Given the following sqlite SQL table definition, write a SQL query
that answers the user's request.

### sqlite SQL table, with its properties:
# Sales({cols})
"""
    return prompt

print(build_schema_prompt(df))

In [ ]:
def prompt_input():
    """
    Collect a natural-language request from the user (CLI input).
    """
    return input("What would you like to analyze?: ")


# Get the user's natural-language request (e.g., "Show total sales by region")
text_request = prompt_input()

# Combine the schema context + the user request into one prompt for the LLM
full_prompt = f"{build_schema_prompt(df)}\nUser request: {text_request}"

# Preview the final prompt (helpful for debugging prompt quality)
full_prompt

In [ ]:
response = client.chat.completions.create(
  model="gpt-5.1",
  messages=[
    {"role": "system", "content": "You are an assistant that generates SQL queries based on the given SQLite table definition and a natural language request. The query should start with 'SELECT' and end with a semicolon (;)."},
    {"role": "user", "content": f"A query to answer: {full_prompt}"},
  ],

    # Control how much "thinking" the model is allowed to do.
    # gpt-5.1 supports: "none" | "low" | "medium" | "high"
    reasoning_effort="none",

    # Sampling controls (ONLY supported when reasoning_effort="none" for GPT-5.1 / GPT-5.2).
    # Use temperature=0 for more deterministic SQL.
    temperature=0,
    # top_p=1,  # Alternative to temperature (do not set both temperature and top_p)

    # Upper bound for tokens generated (includes visible output tokens + reasoning tokens).
    max_completion_tokens = 1000
)


# Extract only the generated SQL text from the first completion choice.
# (This is the raw assistant output before any post-processing/validation.)
query_text = response.choices[0].message.content

In [ ]:
def normalize_sql_response(response):
    """
    Normalize the model output into a safe, executable SQLite SELECT statement.

    What this function does:
    - Reads the model-generated text from the API response
    - Trims whitespace and normalizes formatting
    - Ensures the query is a SELECT statement (basic guardrail)
    - Ensures the query ends with a semicolon (;)
    """

    # Extract the assistant's generated text (SQL) and remove leading/trailing whitespace
    query = response.choices[0].message.content.strip()

    # Basic safety/consistency check:
    # If the model didn't start with "SELECT", prepend it so the output is executable SQL.
    # (Note: for stricter safety, you could raise an error instead of auto-fixing.)
    if not query.upper().startswith("SELECT"):
        query = "SELECT " + query

    # Ensure the SQL statement ends with a semicolon (common requirement for execution/logging)
    if not query.endswith(";"):
        query += ";"

    # Return the normalized SQL query string
    return query


print(normalize_sql_response(response))

In [ ]:
result = temp_db.execute(normalize_sql_response(response))
print(result.fetchall())
